In [1]:
from appgym.web_env import WebEnv

In [2]:
env = WebEnv("http://127.0.0.1:8080/", content_selector='.mobile-content')

In [3]:
state, actions = env.reset()

In [4]:
state.image.shape

(480, 300, 3)

In [5]:
len(actions)

360

In [5]:
480 / 20

24.0

In [6]:
300 / 20

15.0

In [7]:
24*15

360

In [8]:
480 / 8

60.0

In [9]:
env._viewport()

Rect(x=250, y=60, width=300, height=480)

In [10]:
range(10)

range(0, 10)

In [12]:
list(range(1, 11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]